In [65]:
from pathlib import Path

from data import TradeCapacityMapFacilitator, StackedQuantityEvolutionFacilitator, StackedQuantityEvolutionFacilitatorBase

In [66]:
CURRENT_DIR = Path().resolve()
SOLUTION_FILEPATH = CURRENT_DIR / "Solution_40_12_day.sol"
TITLE_ADDON = " - Day"

### Trade capacities between European countries 2018 and 2050

In [67]:
trade_capacity_plotter = TradeCapacityMapFacilitator(sol_path=SOLUTION_FILEPATH, year=2018)

relevant_data = trade_capacity_plotter.get_relevant_data()

relevant_data.TotalTradeCapacity.filter_by_identifier(column="Fuel", identifier="Power")

fig = trade_capacity_plotter.plot(relevant_data)

fig.layout.title = "Power Trade Capacity in 2018" + TITLE_ADDON

fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))

fig.write_image("PowerTradeCapacity2018" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

No unit conversion applied!


In [68]:
trade_capacity_plotter = TradeCapacityMapFacilitator(sol_path=SOLUTION_FILEPATH, year=2050)

relevant_data = trade_capacity_plotter.get_relevant_data()
print(relevant_data.TotalTradeCapacity.df["Year"].unique())

relevant_data.TotalTradeCapacity.filter_by_identifier(column="Fuel", identifier="Power")

fig = trade_capacity_plotter.plot(relevant_data)

fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))

fig.layout.title = "Power Trade Capacity in 2050" + TITLE_ADDON

fig.write_image("PowerTradeCapacity2050.pdf" + TITLE_ADDON.replace(" ", "") + ".png")

fig.show()

No unit conversion applied!
[2050]


### Storage capacities

In [69]:
storage_capacity_plotter = StackedQuantityEvolutionFacilitator(sol_path=SOLUTION_FILEPATH, region="DE")

relevant_data = storage_capacity_plotter.get_relevant_data()
#relevant_data.filter_by_containing_string(column="Technology", identifier="D_")
relevant_data.filter_by_list(column="Technology", by_filter=["D_Heat_HLDH", "D_Heat_HLI", "D_Heat_HLB"])

fig = storage_capacity_plotter.plot(relevant_data)

fig.update_layout(
    width=1400,
    height=800,
    title="Storage Capacity Germany" + TITLE_ADDON,
    yaxis_title="GW",
    xaxis_title="Year",
)

fig.write_image("StorageProductionGermany.pdf" + TITLE_ADDON.replace(" ", "") + ".png")
fig.show()

No unit conversion applied!


In [70]:
storage_capacity_plotter = StackedQuantityEvolutionFacilitator(sol_path=SOLUTION_FILEPATH, region="All")

relevant_data = storage_capacity_plotter.get_relevant_data()
#relevant_data.filter_by_containing_string(column="Technology", identifier="D_")
relevant_data.filter_by_list(column="Technology", by_filter=["D_Heat_HLDH", "D_Heat_HLI", "D_Heat_HLB"])
relevant_data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name="All", column_to_sum="Value")

fig = storage_capacity_plotter.plot(relevant_data)

fig.update_layout(
    width=1400,
    height=800,
    title="Storage Capacity All Regions" + TITLE_ADDON,
    yaxis_title="GW",
    xaxis_title="Year",
)

fig.write_image("StorageProductionAllRegions.pdf" + TITLE_ADDON.replace(" ", "") + ".png")
fig.show()

No unit conversion applied!
